In [10]:
from utils.cluster_params import *
from utils.constant import *
import numpy as np

target_deps_for_kernel_matrix = generate_itr_to_target_deps_by_schedule_vector(1, RESNET50_ORIGIN_DEPS_FLATTENED, RESNET50_INTERNAL_KERNEL_IDXES)
print(target_deps_for_kernel_matrix)
target_deps_for_decay_matrix = generate_itr_to_target_deps_by_schedule_vector(0, RESNET50_ORIGIN_DEPS_FLATTENED, RESNET50_INTERNAL_KERNEL_IDXES)
print(target_deps_for_decay_matrix)

[  64   64   64  256  256   64   64  256   64   64  256  128  128  512
  512  128  128  512  128  128  512  128  128  512  256  256 1024 1024
  256  256 1024  256  256 1024  256  256 1024  256  256 1024  256  256
 1024  512  512 2048 2048  512  512 2048  512  512 2048]
[64  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1]


In [11]:
from utils.model_utils import *
import torchvision

model = torchvision.models.resnet50(pretrained=True)
engine = ModelUtils(local_rank=0, for_eval=True)
engine.register_state(model=model)
kernel_namedvalue_list = engine.get_all_conv_kernel_namedvalue_as_list()
# print(kernel_namedvalue_list)

In [15]:
pacesetter_dict = {
        4: 4,3: 4,7: 4,10: 4,14: 14,
        13: 14,17: 14,20: 14,23: 14,27: 27,26: 27,30: 27,33: 27,36: 27,
        39: 27,42: 27,46: 46,45: 46,49: 46,52: 46
    }
# 生成 layer_idx_to_clusters 为了计算 kernel matrix
layer_idx_to_clusters_for_kernel_matrix = get_layer_idx_to_clusters(kernel_namedvalue_list=kernel_namedvalue_list,
                                                                  target_deps=target_deps_for_kernel_matrix,
                                                                  pacesetter_dict=pacesetter_dict)

if pacesetter_dict is not None:
    for follower_idx, pacesetter_idx in pacesetter_dict.items():
        # 这里将残差的最后一层的剪枝方案直接等同于直连的剪枝方案
        if pacesetter_idx in layer_idx_to_clusters_for_kernel_matrix:
            layer_idx_to_clusters_for_kernel_matrix[follower_idx] = layer_idx_to_clusters_for_kernel_matrix[pacesetter_idx]
            
print(layer_idx_to_clusters_for_kernel_matrix.keys())
print(layer_idx_to_clusters_for_kernel_matrix[1])

# 生成 layer_idx_to_clusters 为了计算 decay matrix
layer_idx_to_clusters_for_decay_matrix = get_layer_idx_to_clusters(kernel_namedvalue_list=kernel_namedvalue_list,
                                                                  target_deps=target_deps_for_decay_matrix,
                                                                  pacesetter_dict=pacesetter_dict)

if pacesetter_dict is not None:
    for follower_idx, pacesetter_idx in pacesetter_dict.items():
        # 这里将残差的最后一层的剪枝方案直接等同于直连的剪枝方案
        if pacesetter_idx in layer_idx_to_clusters_for_decay_matrix:
            layer_idx_to_clusters_for_decay_matrix[follower_idx] = layer_idx_to_clusters_for_decay_matrix[pacesetter_idx]

print(layer_idx_to_clusters_for_decay_matrix.keys())
print(layer_idx_to_clusters_for_decay_matrix[1])

dict_keys([0, 1, 2, 4, 5, 6, 8, 9, 11, 12, 14, 15, 16, 18, 19, 21, 22, 24, 25, 27, 28, 29, 31, 32, 34, 35, 37, 38, 40, 41, 43, 44, 46, 47, 48, 50, 51, 3, 7, 10, 13, 17, 20, 23, 26, 30, 33, 36, 39, 42, 45, 49, 52])
[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32], [33], [34], [35], [36], [37], [38], [39], [40], [41], [42], [43], [44], [45], [46], [47], [48], [49], [50], [51], [52], [53], [54], [55], [56], [57], [58], [59], [60], [61], [62], [63]]
dict_keys([0, 1, 2, 4, 5, 6, 8, 9, 11, 12, 14, 15, 16, 18, 19, 21, 22, 24, 25, 27, 28, 29, 31, 32, 34, 35, 37, 38, 40, 41, 43, 44, 46, 47, 48, 50, 51, 3, 7, 10, 13, 17, 20, 23, 26, 30, 33, 36, 39, 42, 45, 49, 52])
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 5

In [14]:
deps = RESNET50_ORIGIN_DEPS_FLATTENED  # resnet50 的 通道数量
param_name_to_merge_matrix = generate_merge_matrix_for_kernel(deps=deps,
                                                                      layer_idx_to_clusters=layer_idx_to_clusters_for_kernel_matrix,
                                                                      kernel_namedvalue_list=kernel_namedvalue_list)
add_vecs_to_merge_mat_dicts(param_name_to_merge_matrix)
# print(param_name_to_merge_matrix.keys())
print(param_name_to_merge_matrix['layer1.0.conv1.weight'])

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]], device='cuda:0')


In [16]:
deps = RESNET50_ORIGIN_DEPS_FLATTENED  # resnet50 的 通道数量
param_name_to_decay_matrix = generate_decay_matrix_for_kernel_and_vecs(
            deps=deps,
            layer_idx_to_clusters=layer_idx_to_clusters_for_decay_matrix,
            kernel_namedvalue_list=kernel_namedvalue_list,
            weight_decay=1e-4,
            weight_decay_bias=0,
            centri_strength=0.001)
# print(param_name_to_decay_matrix.keys())
print(param_name_to_decay_matrix['layer1.0.conv1.weight'])

tensor([[ 1.0844e-03, -1.5625e-05, -1.5625e-05,  ..., -1.5625e-05,
         -1.5625e-05, -1.5625e-05],
        [-1.5625e-05,  1.0844e-03, -1.5625e-05,  ..., -1.5625e-05,
         -1.5625e-05, -1.5625e-05],
        [-1.5625e-05, -1.5625e-05,  1.0844e-03,  ..., -1.5625e-05,
         -1.5625e-05, -1.5625e-05],
        ...,
        [-1.5625e-05, -1.5625e-05, -1.5625e-05,  ...,  1.0844e-03,
         -1.5625e-05, -1.5625e-05],
        [-1.5625e-05, -1.5625e-05, -1.5625e-05,  ..., -1.5625e-05,
          1.0844e-03, -1.5625e-05],
        [-1.5625e-05, -1.5625e-05, -1.5625e-05,  ..., -1.5625e-05,
         -1.5625e-05,  1.0844e-03]], device='cuda:0')


dict_keys(['layer1.0.conv1.weight', 'layer1.0.conv2.weight', 'layer1.0.downsample.0.weight', 'layer1.1.conv1.weight', 'layer1.1.conv2.weight', 'layer1.2.conv1.weight', 'layer1.2.conv2.weight', 'layer2.0.conv1.weight', 'layer2.0.conv2.weight', 'layer2.0.downsample.0.weight', 'layer2.1.conv1.weight', 'layer2.1.conv2.weight', 'layer2.2.conv1.weight', 'layer2.2.conv2.weight', 'layer2.3.conv1.weight', 'layer2.3.conv2.weight', 'layer3.0.conv1.weight', 'layer3.0.conv2.weight', 'layer3.0.downsample.0.weight', 'layer3.1.conv1.weight', 'layer3.1.conv2.weight', 'layer3.2.conv1.weight', 'layer3.2.conv2.weight', 'layer3.3.conv1.weight', 'layer3.3.conv2.weight', 'layer3.4.conv1.weight', 'layer3.4.conv2.weight', 'layer3.5.conv1.weight', 'layer3.5.conv2.weight', 'layer4.0.conv1.weight', 'layer4.0.conv2.weight', 'layer4.0.downsample.0.weight', 'layer4.1.conv1.weight', 'layer4.1.conv2.weight', 'layer4.2.conv1.weight', 'layer4.2.conv2.weight', 'layer1.0.conv3.weight', 'layer1.1.conv3.weight', 'layer1.2.c